# Creates the fake_news_llama_3_8b_instruct.csv
###### "fake_news_llama_3_8b_instruct.csv" is an artificially generated dataset entirely composed of fake news. The comma-separated values file will later have the column "class" added to it. All entries will have value 0 under the "class" column to indicate it is a fake news.

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from accelerate import Accelerator
import pandas as pd
import subprocess

# Insert your Hugging Face access token here 
access_token = ""
login(access_token)

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
accelerator = Accelerator()

# Load the model with accelerator
model = AutoModelForCausalLM.from_pretrained(model_name, device_map={"":0})
model, tokenizer = accelerator.prepare(model, tokenizer)

# Function to generate fake news in Brazilian Portuguese
def generate_fake_news(prompt, max_length):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=max_length,
        num_return_sequences=1,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.9,
        pad_token_id=tokenizer.eos_token_id
    )
    fake_news = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return fake_news

categories = {
    "brasil": 300,
    "ciência": 300,
    "entretenimento": 900,
    "mundo": 300,
    "política": 900,
    "saúde": 900
}

# Varied prompts in Brazilian Portuguese for each category(Brazil, Science, Entertainment, World, Politics, Health)
prompts = {
    "brasil": [
        "Escreva uma notícia falsa sobre um evento recente no Brasil.",
        "Escreva uma notícia fictícia sobre um acontecimento inusitado no Brasil.",
        "Escreva uma notícia inventada sobre um escândalo político no Brasil.",
        "Crie uma notícia falsa sobre uma nova lei aprovada no Brasil.",
        "Escreva uma notícia fictícia sobre um desastre natural no Brasil.",
        "Invente uma notícia sobre um festival cultural inédito no Brasil.",
        "Escreva uma notícia falsa sobre um atleta brasileiro que quebrou um recorde mundial.",
        "Crie uma notícia fictícia sobre um movimento social inovador no Brasil.",
        "Invente uma notícia sobre a descoberta de um tesouro histórico no Brasil.",
        "Escreva uma notícia falsa sobre uma mudança radical na economia brasileira."
    ],
    "ciência": [
        "Escreva uma notícia falsa sobre uma recente descoberta científica que chocou o mundo.",
        "Escreva uma notícia fictícia sobre uma nova teoria científica revolucionária.",
        "Escreva uma notícia inventada sobre um experimento científico controverso.",
        "Crie uma notícia falsa sobre a descoberta de uma nova espécie animal.",
        "Escreva uma notícia fictícia sobre uma invenção tecnológica inovadora.",
        "Invente uma notícia sobre a descoberta de uma cura para uma doença incurável.",
        "Escreva uma notícia falsa sobre uma viagem espacial inédita.",
        "Crie uma notícia fictícia sobre a criação de uma inteligência artificial avançada.",
        "Invente uma notícia sobre uma descoberta arqueológica impressionante.",
        "Escreva uma notícia falsa sobre um prêmio Nobel que gerou polêmica."
    ],
    "entretenimento": [
        "Escreva uma notícia falsa sobre uma celebridade ou evento de entretenimento.",
        "Escreva uma notícia fictícia sobre um filme ou série que está fazendo sucesso.",
        "Escreva uma notícia inventada sobre uma premiação no mundo do entretenimento.",
        "Crie uma notícia falsa sobre uma banda que anunciou seu retorno inesperado.",
        "Escreva uma notícia fictícia sobre um escândalo envolvendo uma celebridade famosa.",
        "Invente uma notícia sobre um festival de música que foi cancelado.",
        "Escreva uma notícia falsa sobre um casamento secreto de famosos.",
        "Crie uma notícia fictícia sobre uma nova tendência de moda bizarra.",
        "Invente uma notícia sobre um reality show inovador que está fazendo sucesso.",
        "Escreva uma notícia falsa sobre um novo álbum de um cantor famoso que chocou os fãs."
    ],
    "mundo": [
        "Escreva uma notícia falsa sobre um evento recente no mundo.",
        "Escreva uma notícia fictícia sobre uma crise internacional.",
        "Escreva uma notícia inventada sobre um acordo entre países.",
        "Crie uma notícia falsa sobre uma descoberta importante em um país estrangeiro.",
        "Escreva uma notícia fictícia sobre um conflito militar em uma região específica.",
        "Invente uma notícia sobre uma nova aliança entre países rivais.",
        "Escreva uma notícia falsa sobre um evento esportivo internacional.",
        "Crie uma notícia fictícia sobre um avanço tecnológico em um país estrangeiro.",
        "Invente uma notícia sobre uma catástrofe ambiental em outro continente.",
        "Escreva uma notícia falsa sobre um grande evento cultural em um país estrangeiro."
    ],
    "política": [
        "Escreva uma notícia falsa sobre um evento recente na política.",
        "Escreva uma notícia fictícia sobre uma eleição controversa.",
        "Escreva uma notícia inventada sobre um escândalo político.",
        "Crie uma notícia falsa sobre uma nova política governamental.",
        "Escreva uma notícia fictícia sobre uma aliança inesperada entre partidos.",
        "Invente uma notícia sobre uma renúncia chocante de um líder político.",
        "Escreva uma notícia falsa sobre um protesto massivo contra o governo.",
        "Crie uma notícia fictícia sobre uma mudança constitucional controversa.",
        "Invente uma notícia sobre um conflito interno em um partido político.",
        "Escreva uma notícia falsa sobre uma figura política internacional visitando o Brasil."
    ],
    "saúde": [
        "Escreva uma notícia falsa sobre uma recente descoberta ou evento na área da saúde.",
        "Escreva uma notícia fictícia sobre uma nova doença descoberta.",
        "Escreva uma notícia inventada sobre um tratamento revolucionário.",
        "Crie uma notícia falsa sobre uma campanha de vacinação inovadora.",
        "Escreva uma notícia fictícia sobre um avanço significativo na medicina.",
        "Invente uma notícia sobre um surto de uma doença rara.",
        "Escreva uma notícia falsa sobre uma dieta milagrosa que está fazendo sucesso.",
        "Crie uma notícia fictícia sobre a descoberta de um novo medicamento poderoso.",
        "Invente uma notícia sobre uma conferência internacional de saúde polêmica.",
        "Escreva uma notícia falsa sobre uma tecnologia médica que promete revolucionar tratamentos."
    ]
}

#instruction = "Produce the headline, the subheadline, and the content of the news article. The total word count should be 400."
instruction = "Produza o título, o subtítulo, e o conteúdo da notícia. O total de palavras deve ser 400."

fake_news_list = []

# Initialize category_indices
category_indices = {category: 0 for category in categories.keys()}

# Generate fake news for each category
for category, quantity in categories.items():
    # Prints GPU usage
    try:
        gpu_info = subprocess.check_output("nvidia-smi", universal_newlines=True)
        print("\nGPU Information:")
        print(gpu_info)
    except Exception as e:
        print("\nNo GPU found or `nvidia-smi` command not found.")
    for _ in range(quantity):
        index = category_indices[category]
        base_prompt = prompts[category][index]
        category_indices[category] = (index + 1) % len(prompts[category])
        prompt = base_prompt + instruction
        fake_news = generate_fake_news(prompt, max_length=400)
        fake_news_list.append([category, fake_news])


df = pd.DataFrame(fake_news_list, columns=["category", "fake_news"])

df.to_csv('/content/fake_news_llama_3_8b_instruct.csv', index=False)

print("CSV file with generated fake news has been saved.")

from google.colab import files
files.download('/content/fake_news_llama_3_8b_instruct.csv')

Processing complete. Processed file saved to: ./fake_news_llama_3_8b_instructed_v2_processed_latin.csv
